#### Week 1: Vector Search Applications w/ LLMs.  Authored by Chris Sanchez.

# Week 1 - Notebook 3

# Overview
***
Welcome to the final notebook for Week 1! Take a look at all the ground we've covered so far:
- Chunking/splitting
- Vectorization of text
- Combining with metadata
- Saving to disk
- Class Configuration
- Data Indexing
- Keyword search
- Vector search
- OPTIONAL: Searching with Filters

We are now prepared to move on to a very important topic, **Retrieval Evaluation**.  I hope you've noticed that the search results will differ (sometimes slightly, sometimes by a lot) depeding on which search method you used: `keyword_search` or `vector_search`.  As humans, it's fairly easy for us to determine whether the returned search results are relevant to the query that was submitted, (though even here there will be differing opinions on result relevance).  But how do we systematically determine which search method is better in general?  And how do we measure the relative performance of our retrieval system if we change one of it's parameters...for example, changing our embedding model? What about measuring system performance over time as more documents are added to our datastore?

We need a way to evaluate our retrieval system, and this notebook will show you "one way" of doing that.  I say "one way" because there are many ways to approach this problem, and the method I'm showing you is not perfect (if anything it's a bit too conservative).  Ultimately, measuring retrieval performance is hard because it requires a lot of time and effort, and absent any user [click-data](https://en.wikipedia.org/wiki/Click_tracking), requires some form of data labeling.  With the advent of powerful generative LLMs the process of measuring retrieval performance has become much easier. Let's take a look at how that works.

# Retrieval Evaluation - Process
***
Here's a high-level overview of how the Retrieval Evaluation process in this notebook works:

1. Generate a "golden dataset" of query-context pairs.  I used a pseudo-LlamaIndex implementation for this step.  I say "pseudo" implementation because I used LlamaIndex as the backbone, but I had to rewrite significant portions of the dataset generation code because of the opinionated way that LlamaIndex is built. 100 document chunks (contexts) were randomly selected from the Impact Theory corpus and those chunks were then submitted to the `gpt-3.5-turbo` model which generated a query that could be answered by the context.  The output was 100 query-context pairs along with associated doc_ids. 
   - **Assumptions**:
     - The generated query-context pairs are, in fact, relevant to one another i.e. the query can be answered by the context that it's paired with
     - The generated queries are simliar in style and length to the type of queries that end users would ask
2. The golden dataset consists of three primary keys: `corpus`, `relevant_docs`, and `queries`
     - The `corpus` is the original text context/chunk with it's associated `doc_id`
     - The `queries` are the LLM generated queries, one (or more) for each entry in the `corpus`
     - The `relevant_docs` is a simple lookup table linking the `corpus` docs to the generated `queries`
3. We pass the golden dataset into a retrieval evluation function which does the following:
   - Takes in a `retriever` arg (`WeaviateClient`) and a few other configuration params
   - Iterates over all queries in the golden dataset and retrieves search results for each query from Weaviate datastore
   - Extracts all `doc_id` values from the retrieved results
   - Extracts the `doc_id` from the associated `relevant_docs` for each query
   - Checks if the relevant doc_id is in the list of retrieved result doc_ids
   - After all queries are completed a `hit_rate` score and `mrr` score are calculated for the entire golden dataset
   - Writes results to an `eval_results` folder

#### In a Nutshell
Ulitmately, given a golden dataset consisting of queries, relevant docs, and their associated doc_ids, the `retrieval_evaluation` function is checking if the relevant doc_id is found in the list of retrieved results doc_ids, for each query.

#### Problems with this Approach
The problems with this approach are many, I'll cover a few here:
- The **Assumptions** (see section 1 above) about the golden dataset must hold true.  Given that the pairs are generated by `gpt-3.5-turbo`, I think the first assumption will generally be true.  When reviewing the dataset I did find a few questions that were not answerable given the context, but for the most part they were.  The 2nd assumption though, is going to be dependent on your particular search use case.  I think for the purposes of this course, the questions generated are a decent reflection of how someone would query this dataset, and therefore do the job of measuring retriever performance.  But I would always check a real-world query distribution before using an approach like the one presented here.
- This approach is conversative in that there is only "one" right answer.  Either the relevant `doc_id` is in the results list or it isn't.  In reality, there are going to be several documents that could potentially answer the generated query, but we have no way to account for these other relevant documents, unless of course, we want to manually add doc_ids to the golden dataset (and depending on your business case, you may actually want to do that).
- We aren't measuring recall or precision because we aren't classifying other documents as "negatives".  As was just mentioned, the other documents in the results list may or may not be good matches, we just don't know.  Because we don't know, we can't really classify the other documents as "negatives".  So for this approach, we are measuring the ["hit rate"](https://uplimit.com/course/vector-search-apps/admin2/content/session_cln9hzpkl00721aah4hbz06fc/module/module_clo3hmyh0006p12cb3bmygky4) which is simply a count of the number of times that we found a relevant `doc_id` match in the results list and [Mean Reciprocal Rank (MRR)](https://uplimit.com/course/vector-search-apps/admin2/content/session_cln9hzpkl00721aah4hbz06fc/module/module_clo3hmyh0006p12cb3bmygky4).  We're using MRR over other metrics such as Mean Average Precision (MAP) because we are only looking at a [single relevant answer](https://stats.stackexchange.com/questions/127041/mean-average-precision-vs-mean-reciprocal-rank).  Hit rate is a good enough metric for determining if our retriever is retrieving quality results, and MRR will become more important later on when we add a Reranker to the mix.  

In [6]:
#standard library imports
from typing import List, Tuple, Dict, Any
import time
import os

# utilities
from tqdm.notebook import tqdm
from rich import print
from dotenv import load_dotenv
env = load_dotenv('./.env', override=True)

# Assignment 1.3
***
#### Instructions:
* Import the `golden_100.json` dataset using the `from_json` method of the LlamaIndex `EmbeddingQAFinetuneDataset` Class
  - **side note: The `EmbeddingQAFinetuneDataset` Class is the same class used for creating fine-tuning datasets
* Instantiate a new Weaviate Client (Retriever) and set the `class_name` of the Class that you created in Notebook 2
* Evaluate your retriever results using the `retrieval_evaluation` function
* Submit your results in the form of a text file to Uplimit (the function autogenerates a report in the `dir_outpath` directory).

In [4]:
from retrieval_evaluation import calc_hit_rate_scores, calc_mrr_scores, record_results, add_params
from llama_index.finetuning import EmbeddingQAFinetuneDataset
from weaviate_interface import WeaviateClient

#################
##  START CODE ##
#################

# Load QA dataset
golden_dataset = None

# should see 100 queries
print(f'Num queries in Golden Dataset: {len(golden_dataset.queries)}')

### Instantiate Weaviate client and set Class name
# read env vars from local .env file
client = None
class_name = None

#check if WCS instance is live and ready


#################
##  END CODE   ##
#################

#### Once your golden dataset is loaded in memory, you can view its content using dot notation like so: `golden_dataset.queries`, `golden_dataset.corpus`, `golden_dataset.relevant_docs`

# Project 1: Retrieval Evaluation

In [14]:
def retrieval_evaluation(dataset: EmbeddingQAFinetuneDataset, 
                         class_name: str, 
                         retriever: WeaviateClient,
                         retrieve_limit: int=5,
                         chunk_size: int=256,
                         hnsw_config_keys: List[str]=['maxConnections', 'efConstruction', 'ef'],
                         display_properties: List[str]=['doc_id', 'guest', 'content'],
                         dir_outpath: str='./eval_results',
                         include_miss_info: bool=False,
                         user_def_params: Dict[str,Any]=None
                         ) -> Dict[str, str|int|float]:
    '''
    Given a dataset and a retriever evaluate the performance of the retriever. Returns a dict of kw and vector
    hit rates and mrr scores. If inlude_miss_info is True, will also return a list of kw and vector responses 
    and their associated queries that did not return a hit, for deeper analysis. Text file with results output
    is automatically saved in the dir_outpath directory.

    Args:
    -----
    dataset: EmbeddingQAFinetuneDataset
        Dataset to be used for evaluation
    class_name: str
        Name of Class on Weaviate host to be used for retrieval
    retriever: WeaviateClient
        WeaviateClient object to be used for retrieval 
    retrieve_limit: int=5
        Number of documents to retrieve from Weaviate host
    chunk_size: int=256
        Number of tokens used to chunk text. This value is purely for results 
        recording purposes and does not affect results. 
    display_properties: List[str]=['doc_id', 'content']
        List of properties to be returned from Weaviate host for display in response
    dir_outpath: str='./eval_results'
        Directory path for saving results.  Directory will be created if it does not
        already exist. 
    include_miss_info: bool=False
        Option to include queries and their associated kw and vector response values
        for queries that are "total misses"
    user_def_params : dict=None
        Option for user to pass in a dictionary of user-defined parameters and their values.
    '''

    results_dict = {'n':retrieve_limit, 
                    'Retriever': retriever.model_name_or_path, 
                    'chunk_size': chunk_size,
                    'kw_hit_rate': 0,
                    'kw_mrr': 0,
                    'vector_hit_rate': 0,
                    'vector_mrr': 0,
                    'total_misses': 0,
                    'total_questions':0
                    }
    #add hnsw configs and user defined params (if any)
    results_dict = add_params(client, class_name, results_dict, user_def_params, hnsw_config_keys)
    
    start = time.perf_counter()
    miss_info = []
    for query_id, q in tqdm(dataset.queries.items(), 'Queries'):
        results_dict['total_questions'] += 1
        hit = False
        #make Keyword, Vector, and Hybrid calls to Weaviate host
        try:
            kw_response = retriever.keyword_search(request=q, class_name=class_name, limit=retrieve_limit, display_properties=display_properties)
            vector_response = retriever.vector_search(request=q, class_name=class_name, limit=retrieve_limit, display_properties=display_properties)
            
            #collect doc_ids and position of doc_ids to check for document matches
            kw_doc_ids = {result['doc_id']:i for i, result in enumerate(kw_response, 1)}
            vector_doc_ids = {result['doc_id']:i for i, result in enumerate(vector_response, 1)}
            
            #extract doc_id for scoring purposes
            doc_id = dataset.relevant_docs[query_id][0]
     
            #increment hit_rate counters and mrr scores
            if doc_id in kw_doc_ids:
                results_dict['kw_hit_rate'] += 1
                results_dict['kw_mrr'] += 1/kw_doc_ids[doc_id]
                hit = True
            if doc_id in vector_doc_ids:
                results_dict['vector_hit_rate'] += 1
                results_dict['vector_mrr'] += 1/vector_doc_ids[doc_id]
                hit = True
                
            # if no hits, let's capture that
            if not hit:
                results_dict['total_misses'] += 1
                miss_info.append({'query': q, 'kw_response': kw_response, 'vector_response': vector_response})
        except Exception as e:
            print(e)
            continue
    

    #use raw counts to calculate final scores
    calc_hit_rate_scores(results_dict)
    calc_mrr_scores(results_dict)
    
    end = time.perf_counter() - start
    print(f'Total Processing Time: {round(end/60, 2)} minutes')
    record_results(results_dict, chunk_size, dir_outpath=dir_outpath, as_text=True)
    
    if include_miss_info:
        return results_dict, miss_info
    return results_dict

### Run evaluation over golden dataset

In [10]:
#################
##  START CODE ##
#################
results = None

# Conclusion
***

We now have a way of measuring the performance of our system.  This will allow you to make tweaks/changes to the system and then be able to objectively tell whether or not the tweak/change improved or degraded its performance.  Here are a few things to consider going forward:  

- Keep in mind what the ulitmate goal of the system is that you are building.  For this course, we are trying to retrieve the most relevant documents as possible that will effectively address a user query, assuming the information is found within the corpus.  This means that we don't need pages and pages of relevant results, we actually only need the top 3-5, just enough to allow our Reader (the OpenAI LLM) to answer the user query.  This is an important point to be thinking about as you are making changes to the retrieval system.
- Feel free to set the `include_miss_info` param to `True`.  Doing so will return a list of both the keyword and vector responses that did not contain the relevant `doc_id` (a "total_miss" means the `doc_id` was not present in either the `kw_doc_ids` or the `vector_doc_ids`).  Take a look at the style of the queries being asked and compare them with the returned responses.  Why are those responses being returned?  Are they close to the intent of the query?
- Last but not least, you are now free to make changes to your system to improve the `hit_rate` and `mrr` scores.  If it were me, I'd start with switching out to a more performant [embedding model](https://huggingface.co/spaces/mteb/leaderboard).  There will be more opportunities to pick up some low hanging fruit, but we'll have to wait until the following week when hybrid search and Rerankers are introduced.  Whatever you do though, don't change params for the `SentenceSplitter` that you use for chunking the corpus.  Due to the way the golden dataset is derived, it's unfortunately dependent on those original `SentenceSplitter` settings remaining the same across evaluations. That is, of course, unless you want to build out your own golden dataset....

# **OPTIONAL but encouraged... --> Fine-Tuning an Embedding Model
***
#### This exercise won't cost you anything except some time...

#### It is recommended to run this specific section either locally on your machine or on Google Colab. As a proxy, it will take less than 10 minutes to run in a Macbook Air M1.

Aside from switching out your emebdding model to improve your retrieval results, you could also try fine tuning your embedding model (or better yet, switch out your model and then fine tune the new one...👊).  For the longest time, the problem with fine-tuning sentence embedding models was the lack of access to high quality training data.  Generative LLMs can save you days/weeks of time, depending on how large of a dataset you want to create, by automating the process of generating high quality query/context pairs.  In this section we'll go over the step-by-step process of fine-tuning our `all-MiniLM-L6-v2` embedding model from a pre-generated training dataset consisting of only 300 question-context pairs, and then comparing it's retrieval results to our baseline retrieval scores.  I highly encourage trying this method out, I saw a 10+ point jump in `vector_hit_rate` after fine-tuning the baseline model.

### Fine-tune Walkthrough

1. Get baseline retrieval scores (vector Hit Rate, MRR, and total misses) using out-of-the-box baseline model.  You won't know objectively if fine-tuning had any effect if you don't measure the baseline results first.  I know this goes without saying it, but practitioners sometimes want to jump straight into model improvement without first considering their starting point.
2. Collect a training and validation dataset.  This step has already been completed for you, courtesy of `gpt-3.5-turbo`.  LlamaIndex has a great out-of-the-box solution for generating query/context embedding pairs, but it isn't exactly plug and play, so I had to rewrite the function to achieve comptability for our course.  The training dataset consists of queries generated by the LLM that can be answered from the associated context (text chunk).  These pairs were generated using a prompt specifically written for the Impact Theory corpus so the training and validation data (for the most part) are high quality and contextually relevant. 
3. Instantiate a `SentenceTransformersFinetuneEngine` Class written by LlamaIndex which does a great job of abstracting away most of the details invovled in fine-tuning a Sentence Transformer model.
4. Fit the model and set a path where the new model will reside.  I creaed a `models/` directory in the course repo, and included the directory in the `.gitignore` file so that models aren't being pushed with every commit.
5. Create a new dataset (as you learned in Notebook 1) but this time create the embeddings using the new fine-tuned model.
6. Create a new index on Weaviate using the new dataset you just created.
7. Run the `retrieval_evaluation` function again, but this time instantiate your Weaviate client with the new fine-tuned model, but hold all other parameters constant (i.e. don't change any other parameter from the baseline run).
8. Compare the fine-tuned retrieval results to the baseline results 🥳

### Import Training + Valid datasets

In [1]:
training_path = './data/training_data_300.json'
valid_path = './data/validation_data_100.json'

In [7]:
training_set = EmbeddingQAFinetuneDataset.from_json(training_path)
valid_set = EmbeddingQAFinetuneDataset.from_json(valid_path)
num_training_examples = len(training_set.queries)
num_valid_examples = len(valid_set.queries)
print(f'# Training Samples: {num_training_examples}\n# Validation Samples: {num_valid_examples}')

# Training Samples: 300
# Validation Samples: 100

### Wrangle Model output path

In [11]:
#always a good idea to name your fine-tuned so that you can easily identify it,
#especially if you plan on doing multiple training runs with different params
#also probably a good idea to include the # of training samples you are using in the name

model_id = client.model_name_or_path
model_ext = model_id.split('/')[1]
models_dir = './models'
if not os.path.exists('./models'):
    os.makedirs('./models') 
else:
    print(f'{models_dir} already exists')
ft_model_name = f'finetuned-{model_ext}-{num_training_examples}'
model_outpath = os.path.join(models_dir, ft_model_name)

print(f'Model ID: {model_id}')
print(f'Model Outpath: {model_outpath}')

### Instantiate your Fine-tune engine

In [13]:
from llama_index.finetuning import SentenceTransformersFinetuneEngine

finetune_engine = SentenceTransformersFinetuneEngine(
    training_set,
    batch_size=32,
    model_id=model_id,
    model_output_path=model_outpath,
    val_dataset=valid_set,
    epochs=10
)

### Fit the embedding model

In [ ]:
# finetune_engine.finetune()

The `finetune` method will automatically generate the model directory using the `model_output_path` that you define.  Inside the directory will be a copy of the model itself (`pytorch_model.bin`) along with all the other files it needs.  Also in that folder, assuming you provide a `val_dataset` will be an evaluation report in the `eval` directory.  The evaluation report contains several IR metrics that may or may not be useful to you, but it does allow you to compare score improvements with each training epoch.  The new fine-tuned model is loaded through the `SentenceTransformer` class just like any other HuggingFace repo model. 